# Packages & Functions

## Packages

In [16]:
import os
import time
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim.adam import Adam
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Functions

In [17]:
def scailing(x, y):

    ms = MinMaxScaler()
    ss = StandardScaler()

    x_ss = ss.fit_transform(x)
    y_ms = ms.fit_transform(y)

    return x_ss, y_ms

In [18]:
def window_sliding(x, y, iw, ow):
    
    x_ws, y_ws = list(), list()
    for i in range(len(df)):
        x_end = i + iw
        y_end = x_end + ow
        
        if y_end > len(df):
            break
        
        tx = x[i:x_end, :]
        ty = y[x_end:y_end, :]
        
        x_ws.append(tx)
        y_ws.append(ty)
    
    return torch.FloatTensor(x_ws).to(device), torch.FloatTensor(y_ws).to(device)

In [19]:
from torch.utils.data import DataLoader, Dataset

class windowDataset(Dataset):
    def __init__(self, y, input_window, output_window, stride=1):
        #총 데이터의 개수
        L = y.shape[0]
        #stride씩 움직일 때 생기는 총 sample의 개수
        num_samples = (L - input_window - output_window) // stride + 1

        #input과 output : shape = (window 크기, sample 개수)
        X = np.zeros([input_window, num_samples])
        Y = np.zeros([output_window, num_samples])

        for i in np.arange(num_samples):
            start_x = stride*i
            end_x = start_x + input_window
            X[:,i] = y[start_x:end_x]

            start_y = stride*i + input_window
            end_y = start_y + output_window
            Y[:,i] = y[start_y:end_y]

        X = X.reshape(X.shape[0], X.shape[1], 1).transpose((1,0,2))
        Y = Y.reshape(Y.shape[0], Y.shape[1], 1).transpose((1,0,2))
        self.x = X
        self.y = Y
        
        self.len = len(X)
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    def __len__(self):
        return self.len

In [20]:
def plotting(train_loader, test_loader, actual):
    with torch.no_grad():
        train_pred = []
        test_pred = []

        for data in train_loader:
            seq, target = data
            out = model(seq)
            train_pred += out.cpu().numpy().tolist()

        for data in test_loader:
            seq, target = data
            out = model(seq)
            test_pred += out.cpu().numpy().tolist()
      
    total = train_pred + test_pred
    plt.figure(figsize=(20,10))
    plt.plot(np.ones(100)*len(train_pred), np.linspace(0,1,100), '--', linewidth=0.6)
    plt.plot(actual, '--')
    plt.plot(total, 'b', linewidth=0.6)

    plt.legend(['train boundary', 'actual', 'prediction'])
    plt.show()


# Data Processing

## Load Data

In [21]:
data = pd.read_csv('../Data/train.csv')
print(data.dtypes)

Date      object
Open       int64
High       int64
Low        int64
Volume     int64
Close      int64
dtype: object


## Index & Col

In [22]:
df = data.loc[:, ['Date', 'Open']]
df.rename(columns = {'Open':'AC'}, inplace = True)
df['DAC'] = df['AC'] - df['AC'].shift(1)
df['DDAC'] = df['DAC'] - df['DAC'].shift(1)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)

In [23]:
df

,AC,DAC,DDAC
Date,,,
2015-12-16,120,NaN,NaN
2015-12-17,124,4.0,NaN
2015-12-18,121,-3.0,-7.0
2015-12-21,120,-1.0,2.0
2015-12-22,117,-3.0,-2.0
...,...,...,...
2019-10-14,284,-1.0,-20.0
2019-10-15,284,0.0,1.0
2019-10-16,283,-1.0,-1.0


## Null Values

In [24]:
df.isnull().sum()

AC      0
DAC     1
DDAC    2
dtype: int64

In [25]:
df.dropna(inplace = True)
df

,AC,DAC,DDAC
Date,,,
2015-12-18,121,-3.0,-7.0
2015-12-21,120,-1.0,2.0
2015-12-22,117,-3.0,-2.0
2015-12-23,117,0.0,3.0
2015-12-24,118,1.0,1.0
...,...,...,...
2019-10-14,284,-1.0,-20.0
2019-10-15,284,0.0,1.0
2019-10-16,283,-1.0,-1.0


## Split Target

In [84]:
x = df.iloc[:, 0:]
y = df.iloc[:,:1]
print(x)
print(y)

             AC   DAC  DDAC
Date                       
2015-12-18  121  -3.0  -7.0
2015-12-21  120  -1.0   2.0
2015-12-22  117  -3.0  -2.0
2015-12-23  117   0.0   3.0
2015-12-24  118   1.0   1.0
...         ...   ...   ...
2019-10-14  284  -1.0 -20.0
2019-10-15  284   0.0   1.0
2019-10-16  283  -1.0  -1.0
2019-10-17  304  21.0  22.0
2019-10-18  289 -15.0 -36.0

[965 rows x 3 columns]
             AC
Date           
2015-12-18  121
2015-12-21  120
2015-12-22  117
2015-12-23  117
2015-12-24  118
...         ...
2019-10-14  284
2019-10-15  284
2019-10-16  283
2019-10-17  304
2019-10-18  289

[965 rows x 1 columns]


## Scailing

In [86]:
ms = MinMaxScaler()
ss = StandardScaler()

x_ss = ss.fit_transform(x)
y_ms = ms.fit_transform(y)

In [87]:
x_ss, y_ms

(array([[-0.98782528, -0.46905606, -0.69755913],
        [-0.99740329, -0.17321351,  0.20183235],
        [-1.02613732, -0.46905606, -0.19789719],
        ...,
        [ 0.56381226, -0.17321351, -0.09796481],
        [ 0.76495046,  3.0810545 ,  2.20048009],
        [ 0.62128032, -2.24411134, -3.59559834]]),
 array([[0.11764706],
        [0.11470588],
        [0.10588235],
        [0.10588235],
        [0.10882353],
        [0.10588235],
        [0.10882353],
        [0.11176471],
        [0.10294118],
        [0.08235294],
        [0.08529412],
        [0.07058824],
        [0.10294118],
        [0.10294118],
        [0.09117647],
        [0.10294118],
        [0.09705882],
        [0.07352941],
        [0.06176471],
        [0.07647059],
        [0.08235294],
        [0.07941176],
        [0.07058824],
        [0.05588235],
        [0.05588235],
        [0.04705882],
        [0.03823529],
        [0.03235294],
        [0.03235294],
        [0.04411765],
        [0.03235294],
        [

In [88]:
x = x.to_numpy()
y = y.to_numpy()
x, y = window_sliding(x, y, 60, 7)
x_ss, y_ms = window_sliding(x_ss, y_ms, 60, 7)

In [31]:
x_train = x_ss[:650]
y_train = y_ms[:650]
x_test = x_ss[650:]
y_test = y_ms[650:]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

torch.Size([650, 60, 3])
torch.Size([650, 7, 1])
torch.Size([249, 60, 3])
torch.Size([249, 7, 1])


In [32]:
train = torch.utils.data.TensorDataset(x_train, y_train)
test = torch.utils.data.TensorDataset(x_test, y_test)

batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(dataset = test, batch_size = batch_size, shuffle = False)

# Model

## Define Model1

In [61]:
class lstm_encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1):
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(960, 1)
        
    def forward(self, x_input):
        #print('Encoder')
        #print("input_size: ", x_input.shape)
        out, self.hidden = self.lstm(x_input)
        #print("out_size: ", out.shape)
        #print("hidden_size: ", self.hidden[0].shape)
        out = out.reshape(out.shape[0], -1)
        #print("out_size: ", out.shape)
        out = self.fc1(out)
        #print("out_size: ", out.shape)
        
        return out, self.hidden

In [62]:
class lstm_decoder(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers = 1):
        super(lstm_decoder, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size = output_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            batch_first = True)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x_input, encoder_hidden_states):
        #print('Decoder')
        #print("x_input_size: ", x_input.shape)
        x_input = x_input.unsqueeze(-1)
        #print("x_input_size: ", x_input.shape)
        lstm_out, self.hidden = self.lstm(x_input, encoder_hidden_states)
        #print("lstm_out_size: ", lstm_out.shape)
        #print("hidden_size: ", self.hidden[0].shape)
        output = self.linear(lstm_out)
        #print("output_size: ", output.shape)
        
        return output, self.hidden

In [67]:
class lstm_encoder_decoder(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(lstm_encoder_decoder, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.encoder = lstm_encoder(input_size = input_size, hidden_size = hidden_size)
        self.decoder = lstm_decoder(output_size = output_size, hidden_size = hidden_size)

    def forward(self, inputs, targets, target_len, teacher_forcing_ratio):
        batch_size = inputs.shape[0]
        input_size = inputs.shape[2]
        #print("inputs_size: ", inputs.shape)
        outputs = torch.zeros(batch_size, target_len, 1)
        #print("outputs_size: ", outputs.shape)
        #print('')
        
        #print("Encoding")
        _, hidden = self.encoder(inputs)
        #print("_ size: ", _.shape)
        #print("hidden_size: ", hidden[0].shape)
        decoder_input = inputs[:,-1, 0].unsqueeze(-1)
        #print("decoder_input_size: ", decoder_input.shape)
        #print('')
        
        #print('Decoding')
        #원하는 길이가 될 때까지 decoder를 실행한다.
        for t in range(target_len): 
            out, hidden = self.decoder(decoder_input, hidden)
            #print("out_size: ", out.shape)
            out =  out.squeeze(1)
            #print("out_size: ", out.shape)
            
            # teacher forcing을 구현한다.
            # teacher forcing에 해당하면 다음 인풋값으로는 예측한 값이 아니라 실제 값을 사용한다.
            if random.random() < teacher_forcing_ratio:
                decoder_input = targets[:, t, :]
            else:
                decoder_input = out
            #print("out_size: ", out.shape)
            outputs[:,t,:] = out

        return outputs


In [68]:
model = lstm_encoder_decoder(input_size=3, output_size = 1, hidden_size=16).to(device)

In [69]:
learning_rate=0.01
epoch = 1000
optimizer = Adam(model.parameters(), lr = learning_rate)
criterion = nn.MSELoss()

In [70]:
from tqdm import tqdm
import random

model.train()
with tqdm(range(epoch)) as tr:
    for i in tr:
        total_loss = 0.0
        for x,y in train_loader:
            optimizer.zero_grad()
            x = x.to(device).float()
            y = y.to(device).float()
            output = model(x, y, 7, 0.6).to(device)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.cpu().item()
        tr.set_postfix(loss="{0:.5f}".format(total_loss/len(train_loader)))

100%|████████████████████████████████████████████████████████████████| 1000/1000 [01:43<00:00,  9.69it/s, loss=0.00094]


In [89]:
model(x, y, 7, 0.6)

tensor([[[1.3421],
         [1.4359],
         [1.0603],
         ...,
         [0.9523],
         [1.5185],
         [1.0381]],

        [[1.3413],
         [1.4346],
         [1.0603],
         ...,
         [0.9524],
         [1.5187],
         [1.0382]],

        [[1.3397],
         [1.4345],
         [1.0603],
         ...,
         [0.9524],
         [1.5189],
         [1.0382]],

        ...,

        [[1.2372],
         [1.3048],
         [1.0630],
         ...,
         [0.9641],
         [1.5034],
         [1.0349]],

        [[1.2369],
         [1.3066],
         [1.0630],
         ...,
         [0.9640],
         [1.5033],
         [1.0349]],

        [[1.2380],
         [1.3094],
         [1.0629],
         ...,
         [0.9637],
         [1.5033],
         [1.0349]]], grad_fn=<CopySlices>)

In [90]:
x

tensor([[[121.,  -3.,  -7.],
         [120.,  -1.,   2.],
         [117.,  -3.,  -2.],
         ...,
         [ 97.,  -3.,  -5.],
         [ 98.,   1.,   4.],
         [ 98.,   0.,  -1.]],

        [[120.,  -1.,   2.],
         [117.,  -3.,  -2.],
         [117.,   0.,   3.],
         ...,
         [ 98.,   1.,   4.],
         [ 98.,   0.,  -1.],
         [ 99.,   1.,   1.]],

        [[117.,  -3.,  -2.],
         [117.,   0.,   3.],
         [118.,   1.,   1.],
         ...,
         [ 98.,   0.,  -1.],
         [ 99.,   1.,   1.],
         [101.,   2.,   1.]],

        ...,

        [[373.,  -6.,  -4.],
         [370.,  -3.,   3.],
         [366.,  -4.,  -1.],
         ...,
         [268.,   4.,   7.],
         [268.,   0.,  -4.],
         [272.,   4.,   4.]],

        [[370.,  -3.,   3.],
         [366.,  -4.,  -1.],
         [324., -42., -38.],
         ...,
         [268.,   0.,  -4.],
         [272.,   4.,   4.],
         [273.,   1.,  -3.]],

        [[366.,  -4.,  -1.],
       

In [75]:
x_ss

tensor([[[-0.9878, -0.4691, -0.6976],
         [-0.9974, -0.1732,  0.2018],
         [-1.0261, -0.4691, -0.1979],
         ...,
         [-1.2177, -0.4691, -0.4977],
         [-1.2081,  0.1226,  0.4017],
         [-1.2081, -0.0253, -0.0980]],

        [[-0.9974, -0.1732,  0.2018],
         [-1.0261, -0.4691, -0.1979],
         [-1.0261, -0.0253,  0.3018],
         ...,
         [-1.2081,  0.1226,  0.4017],
         [-1.2081, -0.0253, -0.0980],
         [-1.1985,  0.1226,  0.1019]],

        [[-1.0261, -0.4691, -0.1979],
         [-1.0261, -0.0253,  0.3018],
         [-1.0166,  0.1226,  0.1019],
         ...,
         [-1.2081, -0.0253, -0.0980],
         [-1.1985,  0.1226,  0.1019],
         [-1.1794,  0.2706,  0.1019]],

        ...,

        [[ 1.4258, -0.9128, -0.3978],
         [ 1.3971, -0.4691,  0.3018],
         [ 1.3588, -0.6170, -0.0980],
         ...,
         [ 0.4201,  0.5664,  0.7015],
         [ 0.4201, -0.0253, -0.3978],
         [ 0.4585,  0.5664,  0.4017]],

        [[

## Define Model2

In [40]:
class lstm_encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1):
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)
        
    def forward(self, x_input):
        print('Encoder')
        print("input_size: ", x_input.shape)
        out, self.hidden = self.lstm(x_input)
        print("out_size: ", out.shape)
        print("hidden_size: ", self.hidden[0].shape)
        
        return out, self.hidden

In [41]:
class lstm_decoder(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers = 1):
        super(lstm_decoder, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size = output_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            batch_first = True)
        self.lstm2 = nn.LSTM(input_size = 1,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            batch_first = True)
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x_input, encoder_hidden_states):
        print('Decoder')
        print("x_input_size: ", x_input.shape)
        x_input = x_input.unsqueeze(1)
        print("x_input_size: ", x_input.shape)
        if x_input.shape[-1] == 3:
            lstm_out, self.hidden = self.lstm(x_input, encoder_hidden_states)
        if x_input.shape[-1] == 1:
            lstm_out, self.hidden = self.lstm2(x_input, encoder_hidden_states)
        print("lstm_out_size: ", lstm_out.shape)
        print("hidden_size: ", self.hidden[0].shape)
        output = self.fc2(lstm_out)
        print("output_size: ", output.shape)
        
        return output, self.hidden

In [42]:
class lstm_encoder_decoder(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(lstm_encoder_decoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.encoder = lstm_encoder(input_size = input_size, hidden_size = hidden_size)
        self.decoder = lstm_decoder(output_size = input_size, hidden_size = hidden_size)

    def forward(self, inputs, targets, target_len, teacher_forcing_ratio):
        batch_size = inputs.shape[0]
        input_size = inputs.shape[2]
        print("inputs_size: ", inputs.shape)
        outputs = torch.zeros(batch_size, target_len, 1)
        print("outputs_size: ", outputs.shape)
        print('')
        
        print("Encoding")
        _, hidden = self.encoder(inputs)
        print("_ size: ", _.shape)
        print("hidden_size: ", hidden[0].shape)
        decoder_input = inputs[:, -1, :]
        print("decoder_input_size: ", decoder_input.shape)
        print('')
        
        print('Decoding')
        #원하는 길이가 될 때까지 decoder를 실행한다.
        for t in range(target_len): 
            input_size = decoder_input.shape[-1]
            out, hidden = self.decoder(decoder_input, hidden)
            print("out_size: ", out.shape)
            out =  out.squeeze(1)
            print("out_size: ", out.shape)
            
            # teacher forcing을 구현한다.
            # teacher forcing에 해당하면 다음 인풋값으로는 예측한 값이 아니라 실제 값을 사용한다.
            if random.random() < teacher_forcing_ratio:
                decoder_input = targets[:, t, :]
            else:
                decoder_input = out
            print("out_size: ", out.shape)
            outputs[:,t,:] = out

        return outputs


In [43]:
model = lstm_encoder_decoder(input_size = 3, output_size = 1, hidden_size=16).to(device)

In [44]:
learning_rate=0.01
epoch = 2
optimizer = Adam(model.parameters(), lr = learning_rate)
criterion = nn.MSELoss()

In [45]:
from tqdm import tqdm
import random

model.train()
with tqdm(range(epoch)) as tr:
    for i in tr:
        total_loss = 0.0
        for x,y in train_loader:
            optimizer.zero_grad()
            x = x.to(device).float()
            y = y.to(device).float
            output = model(x, y, 7, 0.6).to(device)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.cpu().item()
        tr.set_postfix(loss="{0:.5f}".format(total_loss/len(train_loader)))

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

inputs_size:  torch.Size([64, 60, 3])
outputs_size:  torch.Size([64, 7, 1])

Encoding
Encoder
input_size:  torch.Size([64, 60, 3])
out_size:  torch.Size([64, 60, 16])
hidden_size:  torch.Size([1, 64, 16])
_ size:  torch.Size([64, 60, 16])
hidden_size:  torch.Size([1, 64, 16])
decoder_input_size:  torch.Size([64, 3])

Decoding
Decoder
x_input_size:  torch.Size([64, 3])
x_input_size:  torch.Size([64, 1, 3])
lstm_out_size:  torch.Size([64, 1, 16])
hidden_size:  torch.Size([1, 64, 16])
output_size:  torch.Size([64, 1, 1])
out_size:  torch.Size([64, 1, 1])
out_size:  torch.Size([64, 1])
out_size:  torch.Size([64, 1])
Decoder
x_input_size:  torch.Size([64, 1])
x_input_size:  torch.Size([64, 1, 1])
lstm_out_size:  torch.Size([64, 1, 16])
hidden_size:  torch.Size([1, 64, 16])
output_size:  torch.Size([64, 1, 1])
out_size:  torch.Size([64, 1, 1])
out_size:  torch.Size([64, 1])


TypeError: 'builtin_function_or_method' object is not subscriptable

## Training

In [331]:
model

lstm_encoder_decoder(
  (encoder): lstm_encoder(
    (lstm): LSTM(3, 16, batch_first=True)
  )
  (decoder): lstm_decoder(
    (lstm): LSTM(3, 16, batch_first=True)
    (linear): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [46]:
for x, y in train_loader:
    print(x, y)

tensor([[[-0.9878, -0.4691, -0.6976],
         [-0.9974, -0.1732,  0.2018],
         [-1.0261, -0.4691, -0.1979],
         ...,
         [-1.2177, -0.4691, -0.4977],
         [-1.2081,  0.1226,  0.4017],
         [-1.2081, -0.0253, -0.0980]],

        [[-0.9974, -0.1732,  0.2018],
         [-1.0261, -0.4691, -0.1979],
         [-1.0261, -0.0253,  0.3018],
         ...,
         [-1.2081,  0.1226,  0.4017],
         [-1.2081, -0.0253, -0.0980],
         [-1.1985,  0.1226,  0.1019]],

        [[-1.0261, -0.4691, -0.1979],
         [-1.0261, -0.0253,  0.3018],
         [-1.0166,  0.1226,  0.1019],
         ...,
         [-1.2081, -0.0253, -0.0980],
         [-1.1985,  0.1226,  0.1019],
         [-1.1794,  0.2706,  0.1019]],

        ...,

        [[-1.1794,  0.2706,  0.1019],
         [-1.1794, -0.0253, -0.1979],
         [-1.1890, -0.1732, -0.0980],
         ...,
         [-1.2177, -0.4691, -0.1979],
         [-1.2369, -0.3211,  0.1019],
         [-1.2369, -0.0253,  0.2018]],

        [[

         [-0.7388,  0.2706,  0.1019]]], device='cuda:0') tensor([[[0.1265],
         [0.1294],
         [0.1324],
         [0.1353],
         [0.1324],
         [0.1324],
         [0.1412]],

        [[0.1294],
         [0.1324],
         [0.1353],
         [0.1324],
         [0.1324],
         [0.1412],
         [0.1324]],

        [[0.1324],
         [0.1353],
         [0.1324],
         [0.1324],
         [0.1412],
         [0.1324],
         [0.1324]],

        [[0.1353],
         [0.1324],
         [0.1324],
         [0.1412],
         [0.1324],
         [0.1324],
         [0.1294]],

        [[0.1324],
         [0.1324],
         [0.1412],
         [0.1324],
         [0.1324],
         [0.1294],
         [0.1353]],

        [[0.1324],
         [0.1412],
         [0.1324],
         [0.1324],
         [0.1294],
         [0.1353],
         [0.1412]],

        [[0.1412],
         [0.1324],
         [0.1324],
         [0.1294],
         [0.1353],
         [0.1412],
         [0.1500]],

         [ 0.9374,  1.8977,  1.1012]]], device='cuda:0') tensor([[[0.3206],
         [0.3147],
         [0.3118],
         [0.3294],
         [0.3382],
         [0.3559],
         [0.3676]],

        [[0.3147],
         [0.3118],
         [0.3294],
         [0.3382],
         [0.3559],
         [0.3676],
         [0.3706]],

        [[0.3118],
         [0.3294],
         [0.3382],
         [0.3559],
         [0.3676],
         [0.3706],
         [0.3794]],

        [[0.3294],
         [0.3382],
         [0.3559],
         [0.3676],
         [0.3706],
         [0.3794],
         [0.3853]],

        [[0.3382],
         [0.3559],
         [0.3676],
         [0.3706],
         [0.3794],
         [0.3853],
         [0.3735]],

        [[0.3559],
         [0.3676],
         [0.3706],
         [0.3794],
         [0.3853],
         [0.3735],
         [0.3912]],

        [[0.3676],
         [0.3706],
         [0.3794],
         [0.3853],
         [0.3735],
         [0.3912],
         [0.4000]],

In [52]:
y[:, :, :]

tensor([[[0.8676],
         [0.8912],
         [0.8765],
         [0.8676],
         [0.8206],
         [0.7765],
         [0.7853]],

        [[0.8912],
         [0.8765],
         [0.8676],
         [0.8206],
         [0.7765],
         [0.7853],
         [0.8029]],

        [[0.8765],
         [0.8676],
         [0.8206],
         [0.7765],
         [0.7853],
         [0.8029],
         [0.7176]],

        [[0.8676],
         [0.8206],
         [0.7765],
         [0.7853],
         [0.8029],
         [0.7176],
         [0.7618]],

        [[0.8206],
         [0.7765],
         [0.7853],
         [0.8029],
         [0.7176],
         [0.7618],
         [0.7559]],

        [[0.7765],
         [0.7853],
         [0.8029],
         [0.7176],
         [0.7618],
         [0.7559],
         [0.7529]],

        [[0.7853],
         [0.8029],
         [0.7176],
         [0.7618],
         [0.7559],
         [0.7529],
         [0.8735]],

        [[0.8029],
         [0.7176],
         [0.7618]

In [355]:
input_size

1